# Bicycle Accidents Hotspot (Seattle)

As a dedicated triathlete who frequently takes to the open road to bike, I am acutely aware of the safety concerns associated with sharing the road with motor vehicles. This realization has prompted me to embark on a project aimed at analyzing and identifying areas to avoid during my training sessions and recommending safer alternatives. The motivation behind this endeavor is driven by a genuine concern for the safety of myself and fellow cyclists, be it commuters or athletes.

In [1]:
#list of imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, jaccard_score, log_loss
import warnings

In [2]:
warnings.filterwarnings('ignore')
%matplotlib inline 
pd.set_option('display.max_columns', None)
#open & read file
data = pd.read_csv('Collisions.csv')

## Data Exploration

In [3]:
#Shape -> dimensions of dataset & info -> attributes, datatype of attributes and the number of values
data.shape

(221266, 40)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221266 entries, 0 to 221265
Data columns (total 40 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   X                213797 non-null  float64
 1   Y                213797 non-null  float64
 2   OBJECTID         221266 non-null  int64  
 3   INCKEY           221266 non-null  int64  
 4   COLDETKEY        221266 non-null  int64  
 5   REPORTNO         221266 non-null  object 
 6   STATUS           221266 non-null  object 
 7   ADDRTYPE         217554 non-null  object 
 8   INTKEY           71823 non-null   float64
 9   LOCATION         216680 non-null  object 
 10  EXCEPTRSNCODE    100863 non-null  object 
 11  EXCEPTRSNDESC    11775 non-null   object 
 12  SEVERITYCODE     221265 non-null  object 
 13  SEVERITYDESC     221266 non-null  object 
 14  COLLISIONTYPE    194767 non-null  object 
 15  PERSONCOUNT      221266 non-null  int64  
 16  PEDCOUNT         221266 non-null  int6


Further information about the attributes can be found at the<a href="https://www.seattle.gov/Documents/Departments/SDOT/GIS/Collisions_OD.pdf" target="_blank"> following link</a>. Due to the focus on accidents involving cyclists, we filter data to that involving at least 1 cycle aka PEDCYLCOUNT>0. The below shows the first 3 rows out of 5988 rows.

In [5]:
data = data[data['PEDCYLCOUNT']>0].reset_index()
data.head(3)

,index,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,74,-122.321877,47.605770,75,1361,1361,3593847,Matched,Intersection,29817.0,BOREN AVE AND JEFFERSON ST,,NaN,2,Injury Collision,Cycles,2,0,1,1,1,0,0,2013/04/02 00:00:00+00,4/2/2013 6:42:00 PM,At Intersection (intersection related),18.0,"MOTOR VEHICLE STRUCK PEDALCYCLIST, FRONT END A...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,45,Vehicle - Pedalcyclist,25156,0,N
1,102,-122.318111,47.606206,103,1138,1138,3605914,Matched,Intersection,29810.0,11TH AVE AND E JEFFERSON ST,,NaN,2,Injury Collision,Cycles,2,0,1,1,1,0,0,2013/03/29 00:00:00+00,3/29/2013 6:10:00 PM,At Intersection (intersection related),18.0,"MOTOR VEHICLE STRUCK PEDALCYCLIST, FRONT END A...",NaN,N,Clear,Dry,Daylight,Y,NaN,NaN,45,Vehicle - Pedalcyclist,10453,0,N
2,112,-122.342366,47.622638,113,25000,25000,1777159,Matched,Block,NaN,DEXTER AVE N BETWEEN HARRISON ST AND REPUBLICA...,NaN,NaN,2,Injury Collision,Cycles,2,0,1,1,1,0,0,2004/01/23 00:00:00+00,1/23/2004 8:45:00 AM,Mid-Block (not related to intersection),56.0,PEDALCYCLIST STRUCK MOTOR VEHICLE LEFT SIDE SI...,NaN,0,Unknown,Unknown,Daylight,NaN,4023036.0,NaN,45,Vehicle - Pedalcyclist,0,0,N


In [6]:
#Missing Value %
pd.DataFrame(data = [round(i/len(data) * 100, 2) for i in data.isna().sum().to_list()],
            index = data.columns,
            columns = ['Missing Values (%)']).T

,index,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
Missing Values (%),0.0,0.67,0.67,0.0,0.0,0.0,0.0,0.0,0.07,42.5,0.2,43.37,98.5,0.02,0.0,0.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.73,0.0,0.0,84.22,0.0,0.1,0.08,0.12,68.4,49.13,97.71,0.28,0.28,0.0,0.0,0.0


## Data Cleaning

First, features relevant to our analysis would be chosen and analyzed. Based on definition of our problem, attributes that will influence our analysis are:
* **Location:** Latitude (X), Longitude (Y), Address Type (ADDRTYPE)
* **Severity:** A code that corresponds to the severity of the collision (SEVERITYCODE), a detailed description of the severity of the collision (SEVERITYDESC)
* **Person Count:** Total number of people involved (PERSONCOUNT), number of bicycles involved in the collision (PEDCYLCOUNT)
* **Date:** The date and time of the incident (INCDTTM)
* **Condition:** Description of the weather conditions (WEATHER), condition of the road (ROADCOND), light conditions during the collision (LIGHTCOND)

In [7]:
df = data[['SEVERITYCODE', 'SEVERITYDESC', 'X', 'Y', 'ADDRTYPE', 'PERSONCOUNT', 'INCDTTM', 'WEATHER', 'ROADCOND', 'LIGHTCOND']]
df.head(3)

,SEVERITYCODE,SEVERITYDESC,X,Y,ADDRTYPE,PERSONCOUNT,INCDTTM,WEATHER,ROADCOND,LIGHTCOND
0,2,Injury Collision,-122.321877,47.605770,Intersection,2,4/2/2013 6:42:00 PM,Clear,Dry,Daylight
1,2,Injury Collision,-122.318111,47.606206,Intersection,2,3/29/2013 6:10:00 PM,Clear,Dry,Daylight
2,2,Injury Collision,-122.342366,47.622638,Block,2,1/23/2004 8:45:00 AM,Unknown,Unknown,Daylight


Subsequently, we need to handle the missing values since missing values will interfere with the prediction and analysis results. This can be done ​​by deleting or filling them in. Since the missing values are minimal (less than 1%), we can delete the missing values instead.

In [8]:
pd.DataFrame(data = [round(i/len(df) * 100, 2) for i in df.isna().sum().to_list()],
            index = df.columns,
            columns = ['Missing Values (%)']).T

,SEVERITYCODE,SEVERITYDESC,X,Y,ADDRTYPE,PERSONCOUNT,INCDTTM,WEATHER,ROADCOND,LIGHTCOND
Missing Values (%),0.02,0.0,0.67,0.67,0.07,0.0,0.0,0.1,0.08,0.12


In [9]:
df = df.dropna()

In [10]:
pd.DataFrame(data = [round(i/len(df) * 100, 2) for i in df.isna().sum().to_list()],
            index = df.columns,
            columns = ['Missing Values (%)']).T

,SEVERITYCODE,SEVERITYDESC,X,Y,ADDRTYPE,PERSONCOUNT,INCDTTM,WEATHER,ROADCOND,LIGHTCOND
Missing Values (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next, we need to detect and remove duplicate values that interfere with the analysis and prediction results to make the dataset cleaner.

In [11]:
df.duplicated().sum()

4

In [12]:
df = df.drop_duplicates()
df.duplicated().sum()

0

## Data Analysis